In [ ]:
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K
from concurrent.futures import ThreadPoolExecutor

# Load the ELMo model
elmo = hub.load("https://tfhub.dev/google/elmo/3")

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Function to preprocess the lyrics and generate ELMo embeddings
def elmo_embeddings(lyrics):
    embeddings = elmo(lyrics, signature="default", as_dict=True)["elmo"]
    return K.squeeze(K.slice(embeddings, [0, 0, 0], [-1, -1, -1]), axis=0)

# Function to preprocess the lyrics and generate BERT embeddings
def bert_embeddings(lyrics):
    input_ids = tf.constant(tokenizer.encode(lyrics))[None, :]
    outputs = bert_model(input_ids)
    embeddings = outputs.last_hidden_state
    return K.squeeze(embeddings, axis=0)

# Load your lyrics dataset from the CSV file
dataset = pd.read_csv('lyrics_dataset.csv')

# Preprocess the lyrics column
lyrics = dataset['lyrics'].tolist()

# Use ThreadPoolExecutor to parallelize embedding generation
with ThreadPoolExecutor() as executor:
    # Generate ELMo embeddings
    elmo_embeddings = list(executor.map(elmo_embeddings, lyrics))
    
    # Generate BERT embeddings
    bert_embeddings = list(executor.map(bert_embeddings, lyrics))

# Combine ELMo and BERT embeddings
combined_embeddings = pd.concat([pd.DataFrame(elmo_embeddings), pd.DataFrame(bert_embeddings)], axis=1)

# Add the combined embeddings to your dataset
dataset = pd.concat([dataset, combined_embeddings], axis=1)
